# Transfer Learning Pneumonia Project 12/20/22

## Import Initial Data 

In [2]:
# imports 
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [15]:
# Figure out how to import this model 
# from tensorflow.keras.applications.resnet50v2 import ResNet50V2

ModuleNotFoundError: No module named 'tensorflow.keras.applications.resnet50v2'

In [17]:
# Note: There are only 8 images per label here, so we're going to have to change what we do here 
# Confirmed that the # of images in the 3 folders sum to 5,856, which is the total number of images 

Found 16 files belonging to 2 classes.


In [3]:
# Use validation_split to get the training and validation data 
train_data = keras.utils.image_dataset_from_directory(
    'initial_data/train',
    image_size=(180, 180), 
    batch_size=32, 
    validation_split=0.2, 
    subset = 'training', 
    seed = 10
)

Found 399 files belonging to 2 classes.
Using 320 files for training.


In [4]:
# Validation data: 79 files 
val_data = keras.utils.image_dataset_from_directory(
    'initial_data/train',
    image_size=(180, 180), 
    batch_size=32, 
    validation_split=0.2, 
    subset = 'validation', 
    seed = 10
)

Found 399 files belonging to 2 classes.
Using 79 files for validation.


In [5]:
# Test data: 50 pneumonia and 50 normal images 
test_data = keras.utils.image_dataset_from_directory(
    'initial_data/test',
    image_size=(180, 180), 
    batch_size=32, 
)

Found 100 files belonging to 2 classes.


## Initial ResNet 50 model: 

In [28]:
# Function to build model 
def build_resnet50_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # Resnet 50 basemodel 
    base_model = ResNet50(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [29]:
# Build the model: 
resnet50_mod = build_resnet50_model()

In [31]:
# Function to fit model 
def fit_model(model, train_set, validation_set):
    """Fit a model with the above stated criteria"""
    # Set patience to 5 so it doesn't take too long to fit 
    early_stopping = keras.callbacks.EarlyStopping(patience = 5)
    
    model.fit(train_set, 
              validation_data = validation_set, 
              callbacks = [early_stopping], 
              epochs = 500)
    
    return model

In [33]:
# Fit the model: 
# Started 12:18 pm; ended 12:55 pm --> 7 min to train 
# Best model: 
fitted_resnet50_mod = fit_model(resnet50_mod, train_data, val_data)

Epoch 1/500
10/10 [==============================] - 56s 5s/step - loss: 19.5305 - accuracy: 0.7906 - val_loss: 3.9967 - val_accuracy: 0.9620
Epoch 2/500
10/10 [==============================] - 44s 4s/step - loss: 3.1408 - accuracy: 0.9375 - val_loss: 2.7341 - val_accuracy: 0.9620
Epoch 3/500
10/10 [==============================] - 39s 4s/step - loss: 4.7673 - accuracy: 0.9187 - val_loss: 3.5335 - val_accuracy: 0.9620
Epoch 4/500
10/10 [==============================] - 40s 4s/step - loss: 0.8546 - accuracy: 0.9750 - val_loss: 3.9465 - val_accuracy: 0.9494
Epoch 5/500
10/10 [==============================] - 40s 4s/step - loss: 0.8305 - accuracy: 0.9750 - val_loss: 2.0052 - val_accuracy: 0.9494
Epoch 6/500
10/10 [==============================] - 42s 4s/step - loss: 1.0111 - accuracy: 0.9719 - val_loss: 2.5154 - val_accuracy: 0.9747
Epoch 7/500
10/10 [==============================] - 39s 4s/step - loss: 1.1350 - accuracy: 0.9656 - val_loss: 3.0961 - val_accuracy: 0.9620
Epoch 8/500


## Initial InceptionV3 model: 

In [34]:
# Function to build model 
def build_inceptionV3_model():
    
    # Define input shape for the model 
    inputs = keras.Input(shape = (180, 180, 3))
    # InceptionV3 base model 
    base_model = InceptionV3(input_shape = (180, 180, 3), weights = 'imagenet', include_top = False)
    
    for layer in base_model.layers:
        layer.trainable = False
    
    x = keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
 
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [35]:
# Build the model: 
inceptionV3_mod = build_inceptionV3_model()

87924736/87910968 [==============================] - 5s 0us/step


In [36]:
# Fit the model: 
# Started 12:48 pm; ended pm --> 9 min to train 
fitted_inceptionV3_mod = fit_model(inceptionV3_mod, train_data, val_data)

Epoch 1/500
10/10 [==============================] - 31s 2s/step - loss: 336.5932 - accuracy: 0.6156 - val_loss: 18.4656 - val_accuracy: 0.9241
Epoch 2/500
10/10 [==============================] - 23s 2s/step - loss: 44.6391 - accuracy: 0.8125 - val_loss: 12.4793 - val_accuracy: 0.9114
Epoch 3/500
10/10 [==============================] - 21s 2s/step - loss: 21.8102 - accuracy: 0.7625 - val_loss: 2.8006 - val_accuracy: 0.8734
Epoch 4/500
10/10 [==============================] - 25s 2s/step - loss: 5.3799 - accuracy: 0.8313 - val_loss: 1.1511 - val_accuracy: 0.8987
Epoch 5/500
10/10 [==============================] - 36s 3s/step - loss: 5.0369 - accuracy: 0.8156 - val_loss: 0.9349 - val_accuracy: 0.8101
Epoch 6/500
10/10 [==============================] - 19s 2s/step - loss: 3.2668 - accuracy: 0.8062 - val_loss: 0.4406 - val_accuracy: 0.9241
Epoch 7/500
10/10 [==============================] - 32s 3s/step - loss: 0.8500 - accuracy: 0.7844 - val_loss: 0.1891 - val_accuracy: 0.9114
Epoch 8